In [ ]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-
#audit.py
#Interação através do tags ways
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

osm_file = open("/media/vagner/Seagate Expansion Drive/500Gb/"+
                "DiskExternoVagner/Cursos/CientistaDados/"+
                "Modulo_4/DW-OSM-RMSP/sample.osm", "r")

street_type_re = re.compile(r'^\S+\.?',re.IGNORECASE)
street_types = defaultdict(set)

expected = ["Alameda", "Avenida", "Estrada", "Largo", "Praça", "Rodovia", "Rua", "Travessa"]

mapping = { "Av": "Avenida",
            "Av.": "Avenida",
            "R": "Rua",
            "R.": "Rua", 
            "rua": "Rua",
            "r.": "Rua",
            "r": "Rua"
            }

def audit_street_type(street_types, street_name):
    """     Utiliza expressão regular para verificar o ínicio do campo "street_name" identificando assim o Tipo de 
    Via (Rua, Alamenda, etc)
    """
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            #print street_types
            
def print_sorted_dict(d):
    """     
    """
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v=d[k]
        print "%s: %d" % (k,v)

def is_street_name(elem):
    """     Verifica se o tag é um atributo de endereço de via (addr:street). Retorna um Boolean (True se atributo
    de endereço de via)           
    """
    return (elem.attrib['k'] == "addr:street")

def audit():
    """     Realiza análise baseada no evento "tag de ínicio"; quando este ocorre o elemento em questão é 
    identificado e se for do tipo "way", o método "iter" é chamado; este método realiza uma iteração de todas as 
    subtags (tags aninhadas) do tipo "tag".
    """
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    pprint.pprint(dict(street_types))
    #pprint.sorted_dict(street_types)
    return street_types

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            name = re.sub(street_type_re, mapping[street_type], name)
    return name


def test():
    st_types = audit()
    pprint.pprint(dict(st_types))
    """
    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name
            if name == "West Lexington St.":
                assert better_name == "West Lexington Street"
            if name == "Baldwin Rd.":
                assert better_name == "Baldwin Road"
    """

if __name__ == '__main__':
    test()
    osm_file.close()